## MLflow's Model Registry

In [1]:
from mlflow.tracking import MlflowClient


MLFLOW_TRACKING_URI = "sqlite:///mlflow.db"

### Interacting with the MLflow tracking server

The `MlflowClient` object allows us to interact with...
- an MLflow Tracking Server that creates and manages experiments and runs.
- an MLflow Registry Server that creates and manages registered models and model versions. 

To instantiate it we need to pass a tracking URI and/or a registry URI

In [2]:
client = MlflowClient(tracking_uri=MLFLOW_TRACKING_URI)

client.list_experiments()

AttributeError: 'MlflowClient' object has no attribute 'list_experiments'

In [3]:
client.create_experiment(name="my-cool-experiment")

'3'

Let's check the latest versions for the experiment with id `1`...

In [4]:
from mlflow.entities import ViewType

runs = client.search_runs(
    experiment_ids='1',
    filter_string="metrics.rmse < 7",
    run_view_type=ViewType.ACTIVE_ONLY,
    max_results=5,
    order_by=["metrics.rmse ASC"]
)

In [5]:
for run in runs:
    print(f"run id: {run.info.run_id}, rmse: {run.data.metrics['rmse']:.4f}")

run id: d5ae963446e946409b037d4ac1476e82, rmse: 4.6735
run id: 43fa53ee5d9a40d4b47c8c7cadfda74e, rmse: 5.4312
run id: 4bb363e9f53a412a806100782e1eeb0c, rmse: 5.4312


### Interacting with the Model Registry

In this section We will use the `MlflowClient` instance to:

1. Register a new version for the experiment `nyc-taxi-regressor`
2. Retrieve the latests versions of the model `nyc-taxi-regressor` and check that a new version `4` was created.
3. Transition the version `4` to "Staging" and adding annotations to it.

In [6]:
import mlflow

mlflow.set_tracking_uri(MLFLOW_TRACKING_URI)

In [32]:
run_id = "d5ae963446e946409b037d4ac1476e82"
model_uri = f"runs:/{run_id}/model"
mlflow.register_model(model_uri=model_uri, name="nyc-taxi-regressor")

Registered model 'nyc-taxi-regressor' already exists. Creating a new version of this model...
Created version '3' of model 'nyc-taxi-regressor'.


<ModelVersion: aliases=[], creation_timestamp=1716561819244, current_stage='None', description=None, last_updated_timestamp=1716561819244, name='nyc-taxi-regressor', run_id='d5ae963446e946409b037d4ac1476e82', run_link=None, source='/workspaces/mlops-zoomcamp-own/02-experiment-tracking/mlruns/1/d5ae963446e946409b037d4ac1476e82/artifacts/model', status='READY', status_message=None, tags={}, user_id=None, version=3>

In [33]:
model_name = "nyc-taxi-regressor"
latest_versions = client.get_latest_versions(name=model_name)

for version in latest_versions:
    print(f"version: {version.version}, stage: {version.current_stage}")

version: 1, stage: Production
version: 2, stage: Staging
version: 3, stage: None


/tmp/ipykernel_9958/669935608.py:2: FutureWarning: ``mlflow.tracking.client.MlflowClient.get_latest_versions`` is deprecated since 2.9.0. Model registry stages will be removed in a future major release. To learn more about the deprecation of model registry stages, see our migration guide here: https://mlflow.org/docs/2.13.0/model-registry.html#migrating-from-stages
  latest_versions = client.get_latest_versions(name=model_name)


In [34]:
model_version = 3
new_stage = "Production"
client.transition_model_version_stage(
    name=model_name,
    version=model_version,
    stage=new_stage,
    archive_existing_versions=False
)

/tmp/ipykernel_9958/1716261757.py:3: FutureWarning: ``mlflow.tracking.client.MlflowClient.transition_model_version_stage`` is deprecated since 2.9.0. Model registry stages will be removed in a future major release. To learn more about the deprecation of model registry stages, see our migration guide here: https://mlflow.org/docs/2.13.0/model-registry.html#migrating-from-stages
  client.transition_model_version_stage(


<ModelVersion: aliases=[], creation_timestamp=1716561819244, current_stage='Production', description=None, last_updated_timestamp=1716561830755, name='nyc-taxi-regressor', run_id='d5ae963446e946409b037d4ac1476e82', run_link=None, source='/workspaces/mlops-zoomcamp-own/02-experiment-tracking/mlruns/1/d5ae963446e946409b037d4ac1476e82/artifacts/model', status='READY', status_message=None, tags={}, user_id=None, version=3>

In [22]:
model_version = 2
new_stage = "Staging"
client.transition_model_version_stage(
    name=model_name,
    version=model_version,
    stage=new_stage,
    archive_existing_versions=False
)

/tmp/ipykernel_9958/4090915774.py:3: FutureWarning: ``mlflow.tracking.client.MlflowClient.transition_model_version_stage`` is deprecated since 2.9.0. Model registry stages will be removed in a future major release. To learn more about the deprecation of model registry stages, see our migration guide here: https://mlflow.org/docs/2.13.0/model-registry.html#migrating-from-stages
  client.transition_model_version_stage(


<ModelVersion: aliases=[], creation_timestamp=1716561495834, current_stage='Staging', description=None, last_updated_timestamp=1716561546906, name='nyc-taxi-regressor', run_id='43fa53ee5d9a40d4b47c8c7cadfda74e', run_link=None, source='/workspaces/mlops-zoomcamp-own/02-experiment-tracking/mlruns/1/43fa53ee5d9a40d4b47c8c7cadfda74e/artifacts/model', status='READY', status_message=None, tags={}, user_id=None, version=2>

In [35]:
from datetime import datetime

date = datetime.today().date()
client.update_model_version(
    name=model_name,
    version=model_version,
    description=f"The model version {model_version} was transitioned to {new_stage} on {date}"
)

<ModelVersion: aliases=[], creation_timestamp=1716561819244, current_stage='Production', description='The model version 3 was transitioned to Production on 2024-05-24', last_updated_timestamp=1716561836480, name='nyc-taxi-regressor', run_id='d5ae963446e946409b037d4ac1476e82', run_link=None, source='/workspaces/mlops-zoomcamp-own/02-experiment-tracking/mlruns/1/d5ae963446e946409b037d4ac1476e82/artifacts/model', status='READY', status_message=None, tags={}, user_id=None, version=3>

### Comparing versions and selecting the new "Production" model

In the last section, we will retrieve models registered in the model registry and compare their performance on an unseen test set. The idea is to simulate the scenario in which a deployment engineer has to interact with the model registry to decide whether to update the model version that is in production or not.

These are the steps:

1. Load the test dataset, which corresponds to the NYC Green Taxi data from the month of March 2021.
2. Download the `DictVectorizer` that was fitted using the training data and saved to MLflow as an artifact, and load it with pickle.
3. Preprocess the test set using the `DictVectorizer` so we can properly feed the regressors.
4. Make predictions on the test set using the model versions that are currently in the "Staging" and "Production" stages, and compare their performance.
5. Based on the results, update the "Production" model version accordingly.


**Note: the model registry doesn't actually deploy the model to production when you transition a model to the "Production" stage, it just assign a label to that model version. You should complement the registry with some CI/CD code that does the actual deployment.**

In [26]:
from sklearn.metrics import mean_squared_error
import pandas as pd


def read_dataframe(filename):
    df = pd.read_parquet(filename)

    df.lpep_dropoff_datetime = pd.to_datetime(df.lpep_dropoff_datetime)
    df.lpep_pickup_datetime = pd.to_datetime(df.lpep_pickup_datetime)

    df['duration'] = df.lpep_dropoff_datetime - df.lpep_pickup_datetime
    df.duration = df.duration.apply(lambda td: td.total_seconds() / 60)

    df = df[(df.duration >= 1) & (df.duration <= 60)]

    categorical = ['PULocationID', 'DOLocationID']
    df[categorical] = df[categorical].astype(str)
    
    return df


def preprocess(df, dv):
    df['PU_DO'] = df['PULocationID'] + '_' + df['DOLocationID']
    categorical = ['PU_DO']
    numerical = ['trip_distance']
    train_dicts = df[categorical + numerical].to_dict(orient='records')
    return dv.transform(train_dicts)


def test_model(name, stage, X_test, y_test):
    model = mlflow.pyfunc.load_model(f"models:/{name}/{stage}")
    y_pred = model.predict(X_test)
    return {"rmse": mean_squared_error(y_test, y_pred, squared=False)}

In [27]:
df = read_dataframe("/workspaces/mlops-zoomcamp-own/02-experiment-tracking/hw/data/green_tripdata_2023-03.parquet")

In [42]:
import os

artifact_path = '/workspaces/mlops-zoomcamp-own/02-experiment-tracking/mlruns/1/d5ae963446e946409b037d4ac1476e82/artifacts/preprocessor/preprocessor.b'

if os.path.exists(artifact_path):
    client.download_artifacts(run_id=run_id, path=artifact_path, dst_path='.')
else:
    print(f"The file {artifact_path} does not exist.")

MlflowException: The following failures occurred while downloading one or more artifacts from /workspaces/mlops-zoomcamp-own/02-experiment-tracking/mlruns/1/d5ae963446e946409b037d4ac1476e82/artifacts:
##### File /workspaces/mlops-zoomcamp-own/02-experiment-tracking/mlruns/1/d5ae963446e946409b037d4ac1476e82/artifacts/preprocessor/preprocessor.b #####
Invalid path

In [40]:
client.download_artifacts(run_id=run_id, path='/workspaces/mlops-zoomcamp-own/02-experiment-tracking/mlruns/1/d5ae963446e946409b037d4ac1476e82/artifacts/preprocessor/preprocessor.b', dst_path='/workspaces/mlops-zoomcamp-own/02-experiment-tracking/mlruns/1/d5ae963446e946409b037d4ac1476e82/artifacts/preprocessor')

MlflowException: The following failures occurred while downloading one or more artifacts from /workspaces/mlops-zoomcamp-own/02-experiment-tracking/mlruns/1/d5ae963446e946409b037d4ac1476e82/artifacts:
##### File /workspaces/mlops-zoomcamp-own/02-experiment-tracking/mlruns/1/d5ae963446e946409b037d4ac1476e82/artifacts/preprocessor/preprocessor.b #####
Invalid path

In [43]:
import pickle

with open("/workspaces/mlops-zoomcamp-own/02-experiment-tracking/mlruns/1/d5ae963446e946409b037d4ac1476e82/artifacts/preprocessor/preprocessor.b", "rb") as f_in:
    dv = pickle.load(f_in)

In [44]:
X_test = preprocess(df, dv)

In [45]:
target = "duration"
y_test = df[target].values

In [46]:
%time test_model(name=model_name, stage="Production", X_test=X_test, y_test=y_test)

/home/codespace/.python/current/lib/python3.10/site-packages/mlflow/store/artifact/utils/models.py:32: FutureWarning: ``mlflow.tracking.client.MlflowClient.get_latest_versions`` is deprecated since 2.9.0. Model registry stages will be removed in a future major release. To learn more about the deprecation of model registry stages, see our migration guide here: https://mlflow.org/docs/2.13.0/model-registry.html#migrating-from-stages
  latest = client.get_latest_versions(name, None if stage is None else [stage])


OSError: No such file or directory: '/workspaces/mlops-zoomcamp-own/02-experiment-tracking/mlruns/1/d5ae963446e946409b037d4ac1476e82/artifacts/model/.'

In [47]:
%time test_model(name=model_name, stage="Staging", X_test=X_test, y_test=y_test)

OSError: No such file or directory: '/workspaces/mlops-zoomcamp-own/02-experiment-tracking/mlruns/1/43fa53ee5d9a40d4b47c8c7cadfda74e/artifacts/model/.'

In [49]:
client.transition_model_version_stage(
    name=model_name,
    version=3,
    stage="Production",
    archive_existing_versions=True
)

/tmp/ipykernel_9958/684772386.py:1: FutureWarning: ``mlflow.tracking.client.MlflowClient.transition_model_version_stage`` is deprecated since 2.9.0. Model registry stages will be removed in a future major release. To learn more about the deprecation of model registry stages, see our migration guide here: https://mlflow.org/docs/2.13.0/model-registry.html#migrating-from-stages
  client.transition_model_version_stage(


<ModelVersion: aliases=[], creation_timestamp=1716561819244, current_stage='Production', description='The model version 3 was transitioned to Production on 2024-05-24', last_updated_timestamp=1716562116846, name='nyc-taxi-regressor', run_id='d5ae963446e946409b037d4ac1476e82', run_link=None, source='/workspaces/mlops-zoomcamp-own/02-experiment-tracking/mlruns/1/d5ae963446e946409b037d4ac1476e82/artifacts/model', status='READY', status_message=None, tags={}, user_id=None, version=3>